In [1]:
import torch
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

### Theory

##### Siamese NN

Концептуально - сиамские нейронные сети, это две одинаковые по архитектуре сети, с общим набором весов. Зачем это нужно ?  
Ответ очевиден, если указать еще одну особенность таких сетей - на выходе у них нет функции активации типа софтмакса. Их выход и есть  
вектор, который получился в результате работы сети. То есть на деле это некоторое подобие автокодировщика (на деле два автокодировщика в одной сети),  
в том смысле, что он отображает все входные данные в некоторое латентное пространство. Таким образом, если на вход сети будут поданы два  
разных объекта, условная косинусная мера покажет, что вектора далеки друг от друга -> объекты различны. И наоборот, расстояние между  
схожими объектами в латентном пространстве будет невелико и можем сделать вывод, что это один и тот же объект.  

Рассмотрим пример, когда не обойтись без сиамской нейросети. Допустим, перед нами стоит задача разблокировки двери по фотографии  
лица человека. Наивный подход был бы в том, чтобы решать задачу классификации. Взять датасет с лицами людей и пытаться классифицировать  
всех. Это хороший подход, в случае, если у нас есть несколько тысяч различных фотографий одного и того же человека. Если мы классифируем  
каких то селеб, то для некоторых из них это возможно, условный VGG датасет существует достаточно давно. Но, что если это обычные люди,   
у которых и в помине такого количества фотографий нет. Задача становится нерешаемой. Здесь выходят на сцену сиамские нейросети. Мы скормим  
поступившее фото и его вектор будет близок к одному из фото в датасете


Резюмируем: нам нужно, чтобы сеть выдавала такое кодирование f, чтобы:  
||f(a)-f(b)|| -> 0 если a и b из одного класса  
||f(a)-f(b)|| -> ∞ если a и b из разных классов  